# MA3J8 Approximation Theory and Applications 

## 03 - Algebraic Polynomials




In [ ]:
using SoftGlobalScope, LinearAlgebra, LaTeXStrings, Plots
gr();

### 03-1 - Runge's Phenomenon

We consider the function $f : [-1, 1] \to \mathbb{R}$, 
$$
   f(x) = \frac{1}{1 + 25 x^2}
$$
Note that $f$ is analytic on $[-1,1]$, hence from our work on trigonometric approximation we expect excellent approximation properties. We choose a uniform grid, 
$$
  x_j = -1 + 2j/N, \qquad j = 0, \dots, N
$$
and interpolate $f$ at those grid points. 

In [ ]:
using Polynomials
f(x) = 1 / (1 + 25 * x^2)
N = 10
X = range(-1, stop=1, length=N)
p = polyfit(X, f.(X))
xp = range(-1, stop=1, length=200)
plot(xp, f.(xp), lw=2, label = "f")
plot!(xp, p.(xp), lw=2, label = "p$N")
plot!(X, f.(X), lw=0, m=:o, ms=6, c=2, label = "")

this does not look great. Maybe we just aren't using enough points?

In [ ]:
xp = range(-1, stop=1, length=400)
P = plot(xp, f.(xp), lw=2, label = "f")
for N in [10, 20, 30]
    X = range(-1, stop=1, length=N)
    p = polyfit(X, f.(X))
    plot!(P, xp, p.(xp), lw=2, label = "p$N")
end 
P

Clearly, the approximations **diverge**. This is called the Runge phenomenon. It is by no means an indicator that polynomials are poor basis functions for approximation. For example, let us use a least-squares fit w.r.t. exact function values on a fine grid. 

In [ ]:
xp = range(-1, stop=1, length=400)
P = plot(xp, f.(xp), lw=2, label = "f")
err = []
NN = [10, 20, 30, 40]
for N in NN
    X = range(-1, stop=1, length=N)
    p = polyfit(xp, f.(xp), N)
    plot!(P, xp, p.(xp), lw=2, label = "p$N")
    push!(err, norm(f.(xp) - p.(xp), Inf))
end 
plot(P, plot(NN, err, lw=2, m=:o, yaxis = (:log,), label = ""), layout = (1,2))

We have recovered what looks like exponential convergence! Clearly there is something we need to understand.

### 03-2 Interpolation on Chebyshev Points

In the lecture notes we have motivated the Chebyshev interpolation nodes 
$$
  x_j = \cos(\pi j/ N)
$$
We can now check whether they fix the problem we had with equispaced nodes.

In [ ]:
chebnodes(N) = [ cos(j*π/N) for j = N:-1:0 ]

In [ ]:
xp = range(-1, stop=1, length=400)
P = plot(xp, f.(xp), lw=2, label = "f")
NN = [10, 20, 30, 40]
errcheb = []
errfit = [] 
for N in NN
    X = chebnodes(N)
    pcheb = polyfit(X, f.(X))
    plot!(P, xp, pcheb.(xp), lw=2, label = "p$N")
    pfit = polyfit(xp, f.(xp), N)
    push!(errcheb, norm(f.(xp) - pcheb.(xp), Inf))
    push!(errfit, norm(f.(xp) - pfit.(xp), Inf))    
end 
plot(P, 
     plot(NN, [errfit, errcheb], lw=2, m=:o, 
          label = ["fit", "cheb"], yaxis = (:log,)), 
     layout = (1,2))

This is excellent news. We will start from here and explore this in a lot more detail.

Next, we observe another problem: evaluating the Chebyshev interpolant is numerically unstable! (At least how it is implemented in the `Polynomials.jl` package. We will return to this later.

In [ ]:
NN = 10:4:80
errcheb = []
for N in NN
    X = chebnodes(N)
    pcheb = polyfit(X, f.(X))
    push!(errcheb, norm(f.(xp) - pcheb.(xp), Inf))
end 
plot(NN, errcheb, lw=2, m=:o,  label = "", yaxis = (:log,))

In [ ]:
using ApproxFun

In [ ]:
S = Chebyshev(-1..1)

In [ ]:
p = points(S,10)

In [ ]:
NN = 10:4:80
errcheb = []
for N in NN
    X = points(Chebyshev(-1..1), N)
    pcheb = polyfit(X, f.(X))
    push!(errcheb, norm(f.(xp) - pcheb.(xp), Inf))
end 
plot(NN, errcheb, lw=2, m=:o,  label = "", yaxis = (:log,))

In [ ]:
function bary(f, N, x)
    X = chebnodes(N)
    F = f.(X)
    p = 0.5 * (F[1] ./ (x .- X[1]) + (-1)^N * F[N+1] ./(x .- X[N+1]))
    q = 0.5 * (1.0 ./ (x .- X[1]) + (-1)^N ./ (x .- X[N+1]))
    for n = 0:N
        p += (-1)^n * F[n+1] ./ (x .- X[n+1])
        q += (-1)^n ./ (x .- X[n+1])
    end 
    return p ./ q    
end

In [ ]:
f(x) = sqrt(abs(x))
xp = range(-1+0.0123, stop=1-0.00321, length=4000)
NN = [4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]
errcheb = []
# errfit = [] 
for N in NN
    pcheb = bary(f, N, xp)
    push!(errcheb, norm(f.(xp) - pcheb, Inf))
end 

plot(NN, [errcheb, NN.^(-1/2)], lw=2, m=:o, 
    label=["err", "N^-1/2"], xaxis = (:log,), yaxis = (:log,))